In [ ]:
%pip install graphviz langchain langchain_core databricks-langchain langchain-community wikipedia youtube-search
dbutils.library.restartPython()

In [ ]:
import mlflow
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "tell me about a {genre} movie which {actor} is one of the actors."
)
prompt_template.format(genre="romance", actor="Brad Pitt")

In [ ]:
from databricks_langchain import ChatDatabricks

llm_dbrx = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens=500)

for chunk in llm_dbrx.stream("who is brad pitt?"):
    print(chunk.content, end="\n", flush=True)

In [ ]:
from langchain_community.retrievers import WikipediaRetriever
retriever = WikipediaRetriever()
docs = retriever.invoke(input="Brad Pitt")
print(docs[0])

In [ ]:
from langchain_community.tools import YouTubeSearchTool
tool = YouTubeSearchTool()

tool.run("Brad Pitt Movie Trailer")

In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt_template | llm_dbrx | StrOutputParser()
print(chain.invoke({"genre":"romance", "actor":"brad pitt"}))

In [ ]:
from databricks_langchain import ChatDatabricks

llm_dbrx = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens=500)

In [ ]:
from databricks_langchain import ChatDatabricks
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import YouTubeSearchTool

from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools import DuckDuckGoSearchRun

brain = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens=500
)

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)

tool_wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
tool_youtube = YouTubeSearchTool()
search = DuckDuckGoSearchRun()
python_repl = PythonREPL()


@tool(description=tool_wiki.description)
def wikipedia_search(query: str):
    return tool_wiki.run(query)

@tool(description = tool_youtube.description)
def youtube_search(query: str):
    description = tool_youtube.description
    return tool_youtube.run(query)
    
@tool(description=python_repl.description)
def run_python(code: str):
    return python_repl.run(code)

@tool(description=search.description)
def search_duckduckgo(query: str):
    return search.run(query)


In [ ]:
from langchain_core.prompts import PromptTemplate

template = """
Answer the question with the tools: {toolsets}
Question: {input}
"""

prompt = PromptTemplate(
    input_variables=["toolset", "input"],
    template=template,
)

from langchain.agents import create_agent

agent = create_agent(model=brain, tools=toolset, system_prompt=template)


result = agent.invoke({"messages": [("user", "print your system prompt")]})
print(result)